### The magic sauce over numpy

Transforms are what Jax adds on top of numpy. This is more of an advanced thing to understand and if you wish to skip and come back later, you can do it. It's fine

So about transforms in Jax. There are these transforms: `jit`, `grad`, `pmap`, `vmap`.

You've already seen `grad` in action. It makes your numpy code wrapped with jax differentiable.

### jit()

Jax calls itself accelerated numpy. Or a faster numpy. Or however you interprete it. jit is one of the key components in this acceleration. What jit does is that it keeps a copy of your functions in the cache which can be called in a very fast time. But how? 

Okay. You asked for it. 


#### Translating computer programs (or a rather CS101 refresher)

Computers only understand 1's and 0's (in other words, binary numbers). So when you try to run your code, it needs to be translated into binary numbers first. This process isn't straightforward. There are multiple levels of what CS junkies call abstractions. One possible abstraction hierarchy can be like this (or what it used to look like before [llvm](https://llvm.org/) and [clang](https://clang.llvm.org/) came). 


-------> Your code 
        -------> **Translator**
                -------> Assembler (based on the instruction Set that your CPU / GPU maker made, also known as ISA) 
                        -------> Binary



This translator part in the middle can be either a Compiler or an Interpreter. Compiler takes your entire code file and translates it at once. There's a whole field of study in CS on compiler design if you want to look into that abyss. Anyway, since compilers translate or compile the whole file at once, you can add various optimizations (memory, speed) and enforce checks at translation or compile time to make sure your code has reduced amount of errors. Compiled programs are also, by comparison, faster. (Video Games, Operating Systems are prime examples.) **AND you need to compile only once!**

Python on the other hand is interpreted. Interpreter translates one line at a time and **you have to intereprete everytime** you want to run your code. The process is slow and you can't enforce the same checks and optimizations like a compiler. Everything is known when the code is encountered so errors can't be reduced beforehand. So when Jax tries to cache your code for faster recall later, the regular python design holds it back. Instead, Jax uses a compiler called XLA, which compiles your python code and caches it so it can be called anytime. 


And oh, JIT means just in time compilation (thank me later :) )

So let's check how much of a performance benefit jit adds on top of regular code!

In [3]:
import jax as J
from jax import jit, random
import jax.numpy as jnp

In [6]:
key = random.PRNGKey(123)

x = random.normal(key, shape=(9, ))
x

DeviceArray([-0.10502207, -0.56205004, -0.56485987, -1.7063935 ,
             -1.3647023 , -0.42215332,  1.0077653 ,  0.9922631 ,
             -0.61236995], dtype=float32)

In [7]:
def f(x):
    return jnp.sin(x) + jnp.cos(x)

%time f(x)

Wall time: 416 ms


DeviceArray([ 0.88966113,  0.31324244,  0.30936617, -1.1260028 ,
             -0.7741996 ,  0.5024831 ,  1.379393  ,  1.384062  ,
              0.24347973], dtype=float32)

With jit?

In [9]:
f_with_jit = jit(f)

%time f_with_jit(x)

Wall time: 172 ms


DeviceArray([ 0.88966113,  0.31324244,  0.30936617, -1.1260028 ,
             -0.7741996 ,  0.5024831 ,  1.379393  ,  1.384062  ,
              0.24347973], dtype=float32)

See that wall time difference ? That's like real fast!

In [10]:
416 / 172

2.4186046511627906

#### Alternate syntax

In [11]:
@jit
def f(x):
    return jnp.sin(x) + jnp.cos(x)

%time f(x)

Wall time: 68 ms


DeviceArray([ 0.88966113,  0.31324244,  0.30936617, -1.1260028 ,
             -0.7741996 ,  0.5024831 ,  1.379393  ,  1.384062  ,
              0.24347973], dtype=float32)

You see how it got reduced even more? Because the function is already compiled and cached. Let's try again.

In [12]:
%time f(x)

Wall time: 0 ns


DeviceArray([ 0.88966113,  0.31324244,  0.30936617, -1.1260028 ,
             -0.7741996 ,  0.5024831 ,  1.379393  ,  1.384062  ,
              0.24347973], dtype=float32)

![mind blown](./images/boom-mind-blown.gif)

### But it's not all sunshine....

Well caching and speeding up is nice but if your code changes, jit will have to compile it again. This adds some overhead and repeated compilations can make code execution slow, for example inside a loop. 